In [2]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [3]:
!python -V

Python 3.9.19


In [4]:
import pickle 
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)


In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df



In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')


In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)


In [10]:
# y_pred

array([16.24590642, 26.1347962 , 11.88426424, ..., 11.59533603,
       13.11317847, 12.89999218])

In [12]:
import numpy as np
print(np.std(y_pred))

6.247488852238703

In [15]:
df['ride_id'] = f'{2023:04d}/{3:02d}_' + df.index.astype('str')

In [17]:
# df['ride_id'].head()

0    2023/03_0
1    2023/03_1
2    2023/03_2
3    2023/03_3
4    2023/03_4
Name: ride_id, dtype: object

In [19]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predictions'] = y_pred

In [20]:
print(df_result.shape)

(3316216, 2)

In [23]:
print(df_result.tail())

,ride_id,predictions
3403761,2023/03_3403761,11.952635
3403762,2023/03_3403762,20.049958
3403763,2023/03_3403763,11.595336
3403764,2023/03_3403764,13.113178
3403765,2023/03_3403765,12.899992


In [24]:
output_file = "prediction_results"

In [25]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [27]:
# !ls -lah ./prediction_results

-rw-r--r--@ 1 anuragwaliya  staff    65M Jun 19 02:19 ./prediction_results
